In [1]:
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('/Users/vaigupta/Documents/DS/Equity_Simulator/Production Copy/ShoonyaApi-py'))))
from api_helper import ShoonyaApiPy, get_time
import datetime
import logging
import time
import yaml
import pandas as pd
import warnings
warnings.filterwarnings(action= 'ignore')
#sample
logging.basicConfig(level=logging.ERROR)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
#start of our program
api = ShoonyaApiPy()

#yaml for parameters
with open('cred.yml') as f:
    cred = yaml.load(f, Loader=yaml.FullLoader)
    print(cred)

ret = api.login(userid = cred['uid'], password = cred['pwd'], twoFA=cred['factor2'], vendor_code=cred['vc'], api_secret=cred['app_key'], imei=cred['imei'])


{'uid': 'FA67369', 'pwd': 'Rake@3427', 'factor2': 'AUYPV4182G', 'vc': 'FA67369_U', 'app_key': 'a0294fc8b334fa66926920b167aee41e', 'imei': 'asbd2324'}


In [7]:
#flag to tell us if the websocket is open
socket_opened = False
global current_batch
current_batch=[]

#application callbacks
def event_handler_order_update(message):
    print("order event: " + str(message))

def event_handler_quote_update(message):
    global strr
    print(message)
    order_append(message)
    
def order_append(event):
    data = (event['tk'],event['ts'],event['ft'],event['o'],event['l'],event['h'],event['c'],event['v'])
    # add new data record to batch
    current_batch.append(data)
    return current_batch

def open_callback():
    global socket_opened
    socket_opened = True
    print('app is connected')   
    #api.subscribe('NSE|22', feed_type='d')
    api.subscribe(['NSE|22', 'NSE|760'],feed_type='d')


def get_time(time_string):
    data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')
    return time.mktime(data)

In [8]:
api.start_websocket(order_update_callback=event_handler_order_update, subscribe_callback=event_handler_quote_update, socket_open_callback=open_callback)


app is connected
{'t': 'dk', 'e': 'NSE', 'tk': '22', 'ts': 'ACC-EQ', 'pp': '2', 'ls': '1', 'ti': '0.05', 'lp': '2227.00', 'pc': '0.11', 'ft': '1660127398', 'o': '2224.55', 'h': '2234.80', 'l': '2214.50', 'c': '2224.60', 'ap': '2221.38', 'v': '200902', 'ltq': '84', 'ltt': '15:29:57', 'tbq': '126', 'tsq': '0', 'bp1': '2227.00', 'sp1': '0.00', 'bp2': '0.00', 'sp2': '0.00', 'bp3': '0.00', 'sp3': '0.00', 'bp4': '0.00', 'sp4': '0.00', 'bp5': '0.00', 'sp5': '0.00', 'bq1': '126', 'sq1': '0', 'bq2': '0', 'sq2': '0', 'bq3': '0', 'sq3': '0', 'bq4': '0', 'sq4': '0', 'bq5': '0', 'sq5': '0', 'bo1': '3', 'so1': '0', 'bo2': '0', 'so2': '0', 'bo3': '0', 'so3': '0', 'bo4': '0', 'so4': '0', 'bo5': '0', 'so5': '0', 'uc': '2449.70', 'lc': '2004.30', '52h': '2589.00', '52l': '1900.00', '52hd': '15-NOV-2021', '52ld': '07-MAR-2022', 'toi': '5662500'}
{'t': 'dk', 'e': 'NSE', 'tk': '760', 'ts': 'CGPOWER-EQ', 'pp': '2', 'ls': '1', 'ti': '0.05', 'lp': '230.50', 'pc': '0.59', 'ft': '1660127309', 'o': '230.15', 'h'

In [56]:
api.close_websocket()

In [9]:
current_batch

[('22',
  'ACC-EQ',
  '1660127398',
  '2224.55',
  '2214.50',
  '2234.80',
  '2224.60',
  '200902'),
 ('760',
  'CGPOWER-EQ',
  '1660127309',
  '230.15',
  '228.00',
  '233.85',
  '229.15',
  '1114741')]

order event: {'t': 'om', 'norenordno': '22081000359424', 'uid': 'FA67369', 'actid': 'FA67369', 'exch': 'NSE', 'tsym': 'AARTIDRUGS-EQ', 'trantype': 'B', 'qty': '1', 'prc': '12022.00', 'pcode': 'B', 'remarks': 'my_order_001', 'status': 'PENDING', 'reporttype': 'NewAck', 'prctyp': 'LMT', 'ret': 'DAY', 'exchordid': '', 'dscqty': '0', 'blprc': '10000.00', 'bpprc': '11000.00'}
order event: {'t': 'om', 'norenordno': '22081000359424', 'uid': 'FA67369', 'actid': 'FA67369', 'exch': 'NSE', 'tsym': 'AARTIDRUGS-EQ', 'trantype': 'B', 'qty': '1', 'prc': '12022.00', 'pcode': 'B', 'remarks': 'my_order_001', 'rejreason': "RED:'BO' Orders are disallowed after system square off", 'status': 'REJECTED', 'reporttype': 'Rejected', 'prctyp': 'LMT', 'ret': 'DAY', 'exchordid': '', 'dscqty': '0', 'blprc': '10000.00', 'bpprc': '11000.00'}order event: {'t': 'om', 'norenordno': '22081000359424', 'uid': 'FA67369', 'actid': 'FA67369', 'exch': 'NSE', 'tsym': 'AARTIDRUGS-EQ', 'trantype': 'B', 'qty': '1', 'prc': '12022.0

In [52]:
time = datetime.fromtimestamp(int(strr['ft'])).strftime('%Y-%m-%d %H:%M:%S')

In [10]:
json.loads(({'Time':time,'Open':strr['o']}))

[]

In [10]:
token.append(strr['token'])
openn.append(strr['o'])
low.append(strr['l'])
high.append(strr['h'])
close.append(strr['c'])
volume.append(strr['v'])
stock.append(strr['ts'])

'224.00'

In [51]:
strr

{'t': 'df',
 'e': 'NSE',
 'tk': '760',
 'ft': '1659600605',
 'tbq': '288744',
 'tsq': '584930',
 'bq1': '65',
 'bq4': '218',
 'bo1': '5',
 'bo4': '3'}

In [47]:
if isnull(strr['o']):
    print("Print")

KeyError: 'o'

ERROR:websocket:error from callback <bound method NorenApi.__on_open_callback of <api_helper.ShoonyaApiPy object at 0x7fef13d135c0>>: socket is already closed.


In [36]:
current_batch

[('22', '2233.95'), ('760', '224.00')]

app is connected
{'t': 'dk', 'e': 'NSE', 'tk': '22', 'ts': 'ACC-EQ', 'pp': '2', 'ls': '1', 'ti': '0.05', 'lp': '2236.40', 'pc': '0.39', 'ft': '1659522130', 'o': '2233.95', 'h': '2241.55', 'l': '2204.20', 'c': '2227.65', 'ap': '2223.94', 'v': '288612', 'ltq': '10', 'ltt': '15:40:06', 'tbq': '99', 'tsq': '0', 'bp1': '2236.40', 'sp1': '0.00', 'bp2': '0.00', 'sp2': '0.00', 'bp3': '0.00', 'sp3': '0.00', 'bp4': '0.00', 'sp4': '0.00', 'bp5': '0.00', 'sp5': '0.00', 'bq1': '99', 'sq1': '0', 'bq2': '0', 'sq2': '0', 'bq3': '0', 'sq3': '0', 'bq4': '0', 'sq4': '0', 'bq5': '0', 'sq5': '0', 'bo1': '3', 'so1': '0', 'bo2': '0', 'so2': '0', 'bo3': '0', 'so3': '0', 'bo4': '0', 'so4': '0', 'bo5': '0', 'so5': '0', 'uc': '2460.00', 'lc': '2012.80', '52h': '2589.00', '52l': '1900.00', '52hd': '15-NOV-2021', '52ld': '07-MAR-2022', 'toi': '4837250'}
{'t': 'dk', 'e': 'NSE', 'tk': '760', 'ts': 'CGPOWER-EQ', 'pp': '2', 'ls': '1', 'ti': '0.05', 'lp': '232.05', 'pc': '3.52', 'ft': '1659522512', 'o': '224.00', 'h': 

In [15]:
token.append(strr['token'])
openn.append(strr['o'])
low.append(strr['l'])
high.append(strr['h'])
close.append(strr['c'])
volume.append(strr['v'])
volume.append(strr['ts'])

KeyError: 'token'

In [13]:
strr['tk']

'760'

In [11]:
strr['tk']

'760'

In [79]:
from platform import python_version

print(python_version())

3.7.3
